In [1]:
%load_ext autoreload
%autoreload 2

In [26]:
fp = open("tmp.txt", "w")
fp.close()

In [44]:
import sys
#from lstmppo.trainer import LSTMPPOTrainer
sys.argv = [""]

In [ ]:
#trainer = LSTMPPOTrainer()
#trainer.assert_hidden_state_flow()
#trainer.assert_rollout_deterministic()
#trainer.validate_tbptt()
#trainer.validate_lstm_state_flow()

TBPTT value diff: 6.735324859619141e-06
TBPTT logprob diff: 1.1920928955078125e-05
=== LSTM State-Flow Validation ===
T = 128, B = 1
max |values_rec - stored_values|   : 0.0
max |logprobs_rec - stored_logprobs|: 0.0

[t = 0]
 stored value   : -0.1030530035495758
 recomputed val : -0.1030530035495758
 |diff|         : 0.0
 stored logprob : -0.2273954153060913
 recomputed logp: -0.2273954153060913
 |diff|         : 0.0

[t = 1]
 stored value   : -0.03605862334370613
 recomputed val : -0.03605862334370613
 |diff|         : 0.0
 stored logprob : -0.6379159688949585
 recomputed logp: -0.6379159688949585
 |diff|         : 0.0

[t = 2]
 stored value   : -0.19711937010288239
 recomputed val : -0.19711937010288239
 |diff|         : 0.0
 stored logprob : -0.19436419010162354
 recomputed logp: -0.19436419010162354
 |diff|         : 0.0

[t = 3]
 stored value   : -0.033682409673929214
 recomputed val : -0.033682409673929214
 |diff|         : 0.0
 stored logprob : -0.6413920521736145
 recomputed lo